In [ ]:
# 安装依赖
import os,subprocess

def up_pip():
    bat_folder = 'bat'
    if not os.path.exists(bat_folder):
        os.makedirs(bat_folder)
    
    bat_file_path = os.path.join(bat_folder, 'up.bat')
    with open(bat_file_path, 'w') as bat_file:
        bat_file.write("pip install --upgrade pip")
    subprocess.run([bat_file_path], shell=True)
    
def install_package(package_name):
    bat_folder = 'bat'
    if not os.path.exists(bat_folder):
        os.makedirs(bat_folder)
    bat_file_path = os.path.join(bat_folder, 'install.bat')
    with open(bat_file_path, 'w') as bat_file:
        bat_file.write(f'pip install {package_name}')
    subprocess.run([bat_file_path], shell=True)
    
def list():
    bat_folder = 'bat'
    if not os.path.exists(bat_folder):
        os.makedirs(bat_folder)
    bat_file_path = os.path.join(bat_folder, 'plist.bat')
    with open(bat_file_path, 'w') as bat_file:
        bat_file.write("pip list")
    subprocess.run([bat_file_path], shell=True)

def install_dependency():
    up_pip()
    install_package("bcrypt cryptography argon2-cffi")
    install_package("blake3 pycryptodome")
    list()
    print("安装依赖库成功！接下来您就可以正常使用 SHG Pre 啦")
    
if __name__ == "__main__":
    install_dependency()

In [ ]:
# 导入必要的库
import os
import wmi
import hmac
import time
import blake3
import hashlib
import platform
import subprocess
from Crypto.Hash import RIPEMD
from argon2 import PasswordHasher
from base64 import urlsafe_b64encode
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [ ]:
# 生成盐（Salt）
def generate_salt(length=1024):
    return os.urandom(length)

# 使用盐和 SHA2-256 计算哈希值
def salted_sha2256_hash(data, salt, sn):
    sha2256 = hashlib.sha256()
    sha2256.update(salt + sn.encode('utf-8') + data.encode('utf-8'))
    return sha2256.hexdigest()

# 使用盐和 SHA2-512 计算哈希值
def salted_sha2512_hash(data, salt, sn):
    sha2512 = hashlib.sha512()
    sha2512.update(salt + sn.encode('utf-8') + data.encode('utf-8'))
    return sha2512.hexdigest()

# 使用盐和 SHA3-256 计算哈希值
def salted_sha3256_hash(data, salt, sn):
    sha3256 = hashlib.sha3_256()
    sha3256.update(salt + sn.encode('utf-8') + data.encode('utf-8'))
    return sha3256.hexdigest()

# 使用盐和 SHA3-512 计算哈希值
def salted_sha3512_hash(data, salt, device_sn):
    sha3512 = hashlib.sha3_512()
    sha3512.update(salt + device_sn.encode('utf-8') + data.encode('utf-8'))
    return sha3512.hexdigest()

# 使用盐和 SHA2-384 计算哈希值
def salted_sha2384_hash(data, salt, device_sn):
    sha2384 = hashlib.sha384()
    sha2384.update(salt + device_sn.encode('utf-8') + data.encode('utf-8'))
    return sha2384.hexdigest()

# 使用盐和 SHA3-384 计算哈希值
def salted_sha3384_hash(data, salt, device_sn):
    sha3384 = hashlib.sha3_384()
    sha3384.update(salt + device_sn.encode('utf-8') + data.encode('utf-8'))
    return sha3384.hexdigest()

# 使用盐和 BLAKE3 计算哈希值
def salted_blake3_hash(data, salt, device_sn):
    return blake3.blake3(salt + device_sn.encode('utf-8') + data.encode('utf-8')).hexdigest()

# 使用盐和 BLAKE2b 计算哈希值
def salted_blake2b_hash(data, salt, device_sn):
    blake2b = hashlib.blake2b()
    blake2b.update(salt + device_sn.encode('utf-8') + data.encode('utf-8'))
    return blake2b.hexdigest()

# 使用盐和 BLAKE2s 计算哈希值
def salted_blake2s_hash(data, salt, device_sn):
    blake2s = hashlib.blake2s()
    blake2s.update(salt + device_sn.encode('utf-8') + data.encode('utf-8'))
    return blake2s.hexdigest()

# 使用盐和 PIPEMD 计算哈希值
def salted_pipemd256_hash(data, salt, device_sn):
    hasher = RIPEMD.new()
    hasher.update(salt + device_sn.encode('utf-8') + data.encode('utf-8'))
    return hasher.hexdigest()

In [ ]:
# 获取设备信息
class Hardware:
    # 获取 CPU 序列号
    @staticmethod
    def get_cpu_sn():
        c = wmi.WMI()
        for cpu in c.Win32_Processor():
            # print(cpu.ProcessorId.strip())
            return cpu.ProcessorId.strip()

    # 获取主板序列号
    @staticmethod
    def get_baseboard_sn():
        c = wmi.WMI()
        for board_id in c.Win32_BaseBoard():
            # print(board_id.SerialNumber)
            return board_id.SerialNumber

    # 获取设备序列号
    @staticmethod
    def get_serial():
        return platform.node() 

def con():
    cpusn = Hardware.get_cpu_sn()
    boardsn = Hardware.get_baseboard_sn()
    serial = Hardware.get_serial()
    device_sn = cpusn + boardsn + serial
    salt = generate_salt()
    device_sn = salted_sha3512_hash(device_sn, salt, device_sn)
    return device_sn


In [ ]:
# 通过 Argon2 计算哈希值
def argon2_hash(data):
    ph = PasswordHasher()
    hashed = ph.hash(data)
    return hashed

# 使用 PBKDF2 进行密钥衍生
def pbkdf2_hash(data, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=1024,
        salt=salt,
        iterations=66536,
        backend=default_backend()
    )
    key = kdf.derive(data.encode('utf-8'))
    return urlsafe_b64encode(key).decode('utf-8')

# 生成随机密钥
def generate_key(password: str, salt: bytes):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=66536,
        backend=default_backend()
    )
    return kdf.derive(password.encode())

# 加密数据
def encrypt(data: bytes, key: bytes):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    return iv + encryptor.update(data) + encryptor.finalize()

# 生成 HMAC
def create_hmac(data: bytes, key: bytes):
    return hmac.new(key, data, hashlib.sha256).digest()

In [ ]:
# 综合使用 10层算法 和 AES加密算法 等进行哈希计算
def strong_hash(data, salt):
    # 获取设备信息
    device_sn = con()
    
    # 先计算盐化后的 SHA2-256 哈希值
    sha2256_value = salted_sha2256_hash(data, salt, device_sn)
    
    # 使用 SHA2-256 的结果计算盐化后的 SHA2-512 哈希值
    sha2512_value = salted_sha2512_hash(sha2256_value, salt, device_sn)
    
    # 使用 SHA2-512 的结果计算盐化后的 SHA3-256 哈希值
    sha3256_value = salted_sha3256_hash(sha2512_value, salt, device_sn)
    
    # 使用 SHA3-256 的结果计算盐化后的 SHA3-512 哈希值
    sha3512_value = salted_sha3512_hash(sha3256_value, salt, device_sn)
    
    # 使用 SHA3-512 的结果计算盐化后的 SHA2-384 哈希值
    sha2384_value = salted_sha2384_hash(sha3512_value, salt, device_sn)
    
    # 使用 SHA2-384 的结果计算盐化后的 SHA3-384 哈希值
    sha3384_value = salted_sha3384_hash(sha2384_value, salt, device_sn)
    
    # 使用 SHA3-384 的结果计算盐化后的 BLAKE3 哈希值
    blake3_value = salted_blake3_hash(sha3384_value, salt, device_sn)
    
    # 使用 BLAKE3 的结果计算盐化后的 BLAKE2b 哈希值
    blake2b_value = salted_blake2b_hash(blake3_value, salt, device_sn)
    
    # 使用 BLAKE2b 的结果计算盐化后的 BLAKE2s 哈希值
    blake2s_value = salted_blake2s_hash(blake2b_value, salt, device_sn)
    
    # 使用 BLAKE2s 的结果计算盐化后的 PIPEMD 哈希值
    pipemd256_value = salted_pipemd256_hash(blake2s_value, salt, device_sn)
    
    # 使用 AES 加密算法的 CFB 模式加密 PIPEMD 哈希值
    password = sha3512_value + blake3_value
    key = generate_key(password, salt)
    encrypted_hash = encrypt(pipemd256_value.encode(), key)
    
    # 使用 HMAC-SHA256 计算加密后的哈希值的 HMAC 值
    hmac_value = create_hmac(encrypted_hash, key)
    
    # 使用 HMAC-SHA256 的结果计算盐化后的 Argon2 哈希值
    global then_value
    then_value = urlsafe_b64encode(hmac_value).decode()
    argon2_hash_value = argon2_hash(then_value)
    
    # 使用 Argon2 的结果计算盐化后的 PBKDF2 哈希值
    final_hash_value = pbkdf2_hash(argon2_hash_value, salt)
    return final_hash_value

# 循环计算 哈希值
def cycle_strong_hash(data, iterations=4096):
    salt = generate_salt()  # 生成盐
    
    # 循环多次哈希计算
    for _ in range(iterations):
        end_hash_value = strong_hash(data, salt)
    return end_hash_value  # 返回最终的哈希值


In [ ]:
# 主函数
def main():
    # 介绍
    print("Super Strong Hash Generator Preview ")
    print("\nEnter '!@exit' to quit.")
    
    # 循环
    while True:
        d = input("Enter the data to hash: ")
        d = d.strip()
        if d == '!@exit' or d == '！@exit':
            break
        else:
            try:
                # 调用 cycle_strong_hash 函数计算哈希值
                start_time = time.time()  # 记录开始时间
                super_hash_value = cycle_strong_hash(d, iterations=1)
                end_time = time.time()  # 记录结束时间
                print("Super Strong Hash:", super_hash_value)
                print("Time taken to compute hash:", end_time - start_time, "seconds")
            except Exception as e:
                print("Error:", e)

# 运行主函数
if __name__ == "__main__": 
    main()